# Microsoft Agents example with LiteLLM Proxy and SAP LLMs

## [How Microsoft Agents works](https://learn.microsoft.com/en-us/microsoft-365/agents-sdk/quickstart-python)
## [LiteLLM Proxy set up for SAP models](../proxy_set_up.ipynb)

## Installation

In [8]:
%pip install agent-framework litellm


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Set env variables
Add the following variables from the service key in a file called ".env" and put it in the same folder where you run the notebook:
```
LITELLM_PROXY_API_KEY=sk-1234
PROXY_BASE_URL=http://localhost:4000
```

## Run the Microsoft Agents with LiteLLM and SAP LLMs

In [9]:
import os
from typing import Annotated

import litellm
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from pydantic import Field

AttributeError: module 'mcp.types' has no attribute 'ToolUseContent'

Load your credentials as environment variables.

In [ ]:
litellm.use_litellm_proxy = True
load_dotenv()
api_base = os.getenv("PROXY_BASE_URL")
api_key = os.getenv("LITELLM_PROXY_API_KEY")

Define the agent tool.

In [ ]:
def get_weather(
    city: Annotated[str, Field(description="The location to get weather for")]
) -> str:
    city_normalized = city.lower().replace(" ", "")

    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return f"The weather in {city} is sunny with a temperature of 20°C."

Create agent using OpenAIClient with the proxy credentials and the tool

In [ ]:
agent = ChatAgent(
    chat_client=OpenAIChatClient(model_id="sap/gpt-4o",
                                 api_key=api_key,
                                 base_url=api_base,),
    instructions="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, write a couple sentences for "
                "TV weather report in the city, that will be include small jok",
    name="litellm_agent",
    tools=[get_weather],
)

Define the async function with agent run

In [ ]:
async def tools_example():
    result = await agent.run("What's the weather like in Tokyo?")
    print(result.text)

Run the function

In [ ]:
await tools_example()